A [typical workflow](https://medium.com/planet-stories/a-gentle-introduction-to-gdal-part-4-working-with-satellite-data-d3835b5e2971) to create an image from raw satellite data would be:

* Download data.
* Re-order or assemble bands into the desired order (red, green, blue; or near-infrared, red, green; etc.)
* Increase the resolution with pan-sharpening, if desired.
* Contrast-stretch and color-correct the imagery, either algorithmically or by hand.
* Restore georeferencing information, if necessary.
* Crop, re-project, and re-size image to merge with other data.

GDAL - [Geospatial Data Abstraction Library](https://www.gdal.org/gdal_tutorial.html)

# Getting the Data


[The wildfire in California on Nov. 2018](https://github.com/DataForGood-Norway/FireFromSpace#spotted-fires-from-space)

The steps needed to download the imagery data:

1. Locate on geojson.io to get the coordinates: http://geojson.io/#map=13/36.2200/-118.6188
1. Define an Area of Interest (AOI): AOI is the location/geographical window out of which we want to get data.

![](./images/geojson_io.jpeg)

You get this output

```json
{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -118.71156692504884,
              36.161716102717776
            ],
            [
              -118.54196548461914,
              36.161716102717776
            ],
            [
              -118.54196548461914,
              36.27970720524017
            ],
            [
              -118.71156692504884,
              36.27970720524017
            ],
            [
              -118.71156692504884,
              36.161716102717776
            ]
          ]
        ]
      }
    }
  ]
}
```

In [1]:
from geojson import MultiPoint

ModuleNotFoundError: No module named 'geojson'